In [64]:
import pandas as pd
from datetime import date, datetime
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import glob

In [65]:
file_path = "data/Quarterly files"
csv_files = glob.glob(os.path.join(file_path, '*.csv'))
print(csv_files)

['data/Quarterly files/2023 Profit and Loss Q3.csv']


In [66]:
latest_file = max(csv_files, key=os.path.getmtime)

In [67]:
df = pd.read_csv(latest_file, index_col = False)

In [68]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,BJ WIN HOSPITALITY INC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Profit and Loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"For the Nine Months Ending September 30, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Jul. 2023,Aug. 2023,Sept. 2023,Year to Date,NaN,NaN,NaN


In [69]:
cols_to_remove = ['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
df = df.drop(columns = cols_to_remove)

In [70]:
first_month = df.loc[4, 'Unnamed: 1']
second_month = df.loc[4, 'Unnamed: 2']
third_month = df.loc[4, 'Unnamed: 3']
modcols = {
    'Unnamed: 0' : 'metric_type',
    'Unnamed: 1' : first_month,
    'Unnamed: 2' : second_month,
    'Unnamed: 3' : third_month,
}

df = df.rename(columns = modcols)

In [ ]:
def define_quarters(first_month, second_month, third_month):
    if first_month == 'January' and second_month == 'February' and third_month == 'March':
        return 'Q1

In [71]:
df.head()

,metric_type,Jul. 2023,Aug. 2023,Sept. 2023
0,BJ WIN HOSPITALITY INC.,NaN,NaN,NaN
1,Profit and Loss,NaN,NaN,NaN
2,"For the Nine Months Ending September 30, 2023",NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,Jul. 2023,Aug. 2023,Sept. 2023


In [72]:
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

In [73]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(df)

In [74]:
df.iloc[39, df.columns.get_loc('metric_type')] = 'PAYROLL TAXES ADMIN'

In [75]:
df.iloc[:, 1:4] = df.iloc[:, 1:4].applymap(lambda x: x.replace('(', '-').replace(')','') if isinstance(x, str) else x)
df.iloc[:, 1:4] = df.iloc[:, 1:4].applymap(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x)
df.iloc[:, 1:4] = df.iloc[:, 1:4].astype(float)

In [77]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

                    metric_type  Jul. 2023  Aug. 2023 Sept. 2023
0           MEETING ROOM RENTAL     7325.0     5425.0    8419.68
1                   ROOM RENTAL  314940.99  361154.62  344036.08
2          MISCELLANEOUS INCOME     188.37        0.0        0.0
3                        REFUND        0.0      -58.9        0.0
4                Total Revenues  322454.36  366520.72  352455.76
5              FRONT DESK CLERK   14461.92   18898.57   13120.52
6                   HOUSEKEEPER    32923.0   45085.46   32517.71
7               CAFE ATTENDANTS    7098.01   10582.38    7045.26
8                 PAYROLL TAXES    4318.18    5865.55     4131.9
9        WORKERS COMP INSURANCE    4953.27    4953.27    4953.27
10               LINEN PURCHASE        0.0        0.0    2168.12
11               GUEST SUPPLIES    2409.75    4898.18    7300.31
12             LAUNDRY SUPPLIES        0.0    3040.31    2183.47
13     COMMISSION TRAVEL AGENTS    20135.7   14016.25   16150.85
14        CONTINENTAL BRE

In [81]:
rows_to_sum = ['Total Cost Of Sales', 'OFFICERS SALARIES', 'ADMINISTRATIVE SALARIES', 'PAYROLL TAXES ADMIN', 'COMMERCIAL INSURANCE', 
               'TELEPHONE', 'INTERNET & COMPUTER', 'PROPERTY TAXES', 'AUTO INSURANCE']
df['metric_type'] = df['metric_type'].str.lower().str.strip()
rows_to_sum = [row.lower().strip() for row in rows_to_sum]
temp = df['metric_type'].isin(rows_to_sum)
selectRows = df[temp]
print(selectRows)

                metric_type  Jul. 2023  Aug. 2023 Sept. 2023
35      total cost of sales  161386.91  189391.22  172571.19
37        officers salaries   21666.68   21666.68   21666.68
38  administrative salaries   19268.27    20247.9   18703.33
39      payroll taxes admin    3131.51    3206.45     3088.3
40     commercial insurance        0.0     2370.0        0.0
47                telephone     592.69    1555.04    1567.73
48      internet & computer   11676.87    9096.43    7773.84
55           property taxes        0.0   10950.36        0.0
62           auto insurance     652.66     652.66    3936.54


In [89]:
month_totals = selectRows.iloc[:, 1:4].sum()
# print(month_totals)
total_q3_cost = month_totals.sum()

In [90]:
print(total_q3_cost)

706819.9400000001


In [92]:
roomsSold = input("How many rooms sold?")
# retrieve total rooms sold from VM or NA reports
# e.g: Q3 total rooms = 2974+2863+2853
cPORCurrYr = total_q3_cost/int(roomsSold)
print(f"{date_range}: ${round(cPORCurrYr, 2)}")

How many rooms sold? 8690


NameError: name 'date_range' is not defined